# LinkedIn Connection Cleanup Analysis

This notebook analyzes LinkedIn messages to identify connections that have sent multiple unsolicited messages without any reply from the user. These connections are candidates for removal to clean up the network.


## Import Libraries

Import pandas for data manipulation and analysis.


In [1]:
import pandas as pd

## Load Messages Data

Load the LinkedIn messages export file. This contains all message history including:
- Conversation IDs
- Sender and recipient information
- Message content and dates
- Profile URLs


In [2]:
messages = pd.read_csv('data/messages.csv')

messages.info()
messages.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8416 entries, 0 to 8415
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CONVERSATION ID         8416 non-null   object
 1   CONVERSATION TITLE      380 non-null    object
 2   FROM                    8416 non-null   object
 3   SENDER PROFILE URL      8055 non-null   object
 4   TO                      8415 non-null   object
 5   RECIPIENT PROFILE URLS  8405 non-null   object
 6   DATE                    8416 non-null   object
 7   SUBJECT                 754 non-null    object
 8   CONTENT                 8302 non-null   object
 9   FOLDER                  8413 non-null   object
 10  ATTACHMENTS             277 non-null    object
 11  IS MESSAGE DRAFT        8416 non-null   object
dtypes: object(12)
memory usage: 789.1+ KB


,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER,ATTACHMENTS,IS MESSAGE DRAFT
0,2-ODEyM2RlMTQtZjFlMC00ODMwLTg1NjItZDkwYTExNTQ3...,Sponsored Conversation,LinkedIn Member,NaN,Juan Salas,https://www.linkedin.com/in/jmzlx,2025-11-17 22:05:06 UTC,NaN,"<p class=""spinmail-quill-editor__spin-break"">H...",INBOX,NaN,No
1,2-MzllNzEwNTctNGQ5Yy00NWZkLWI5ZDAtY2ExNjM1NDgx...,Christmas gifts,Simone Volpicelli,https://www.linkedin.com/in/simone-volpicelli-...,Juan Salas,https://www.linkedin.com/in/jmzlx,2025-11-15 08:12:24 UTC,Christmas gifts,"Hi Juan, this Christmas, gift trees to your cl...",INBOX,NaN,No
2,2-ZDBjNTMzNDgtYjJkZC00Yzk4LThmNzktYTNjMjgzN2Q1...,NaN,Agus Sánchez - Headhunting and Recruiting Spec...,https://www.linkedin.com/in/recruitinglatam,Juan Salas,https://www.linkedin.com/in/jmzlx,2025-11-13 14:00:59 UTC,NaN,"Hola Juan, cómo estas? Quería comentarte que d...",INBOX,NaN,No
3,2-ZDBjNTMzNDgtYjJkZC00Yzk4LThmNzktYTNjMjgzN2Q1...,NaN,Agus Sánchez - Headhunting and Recruiting Spec...,https://www.linkedin.com/in/recruitinglatam,Juan Salas,https://www.linkedin.com/in/jmzlx,2025-05-01 20:09:23 UTC,NaN,Hi Juan! Are you currently looking to hire new...,INBOX,NaN,No
4,2-ZDBjNTMzNDgtYjJkZC00Yzk4LThmNzktYTNjMjgzN2Q1...,NaN,Agus Sánchez - Headhunting and Recruiting Spec...,https://www.linkedin.com/in/recruitinglatam,Juan Salas,https://www.linkedin.com/in/jmzlx,2024-06-03 15:09:25 UTC,NaN,"Hi Juan, Are you looking to swiftly add new ta...",INBOX,NaN,No


## Load Connections Data

Load the LinkedIn connections export file. This contains information about all connections including:
- Names and profile URLs
- Company and position
- Connection date

Note: We skip the first 3 rows as they contain header information in the LinkedIn export format.


In [3]:
connections = pd.read_csv('data/Connections.csv', skiprows=3)

connections.info()
connections.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7932 entries, 0 to 7931
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   First Name     7194 non-null   object
 1   Last Name      7194 non-null   object
 2   URL            7194 non-null   object
 3   Email Address  253 non-null    object
 4   Company        7111 non-null   object
 5   Position       7121 non-null   object
 6   Connected On   7932 non-null   object
dtypes: object(7)
memory usage: 433.9+ KB


,First Name,Last Name,URL,Email Address,Company,Position,Connected On
0,Michelle,Shocron,https://www.linkedin.com/in/michelle-shocron,NaN,Continuum,Co-founder and CEO at Continuum,15 Nov 2025
1,David,Terán,https://www.linkedin.com/in/david-ter%C3%A1n-4...,NaN,Liebre.ai,CEO,13 Nov 2025
2,Tami (from Miami),Reiss 🚀,https://www.linkedin.com/in/tamireiss,NaN,Leader Within,Keynote Speaker + Corporate Trainer + Executiv...,29 Oct 2025
3,Nahuel,Segovia,https://www.linkedin.com/in/nahuelsegovia,NaN,Flux IT,DevOps Engineer,29 Oct 2025
4,Karl,Pontius,https://www.linkedin.com/in/karl-pontius-6b7350,NaN,ServTrax,President,29 Oct 2025


## Analyze Message Senders

Group messages by sender to see who sends the most messages. This gives us a quick overview of the most active senders in the inbox.


In [4]:
senders = messages.groupby('FROM').size().sort_values(ascending=False)
senders.head()


FROM
Juan Salas                        696
LinkedIn Member                   298
Andrés Alberto Buitrago Alzate     70
Osvaldo Falabella                  25
Mic Mell                           24
dtype: int64

## Analyze Conversations

Group messages by conversation ID to see which conversations have the most messages. This helps identify threads with high activity.


In [5]:
conversations = messages.groupby('CONVERSATION ID').size().sort_values(ascending=False)
conversations.head()


CONVERSATION ID
2-YzQwNWJkM2ItMWMxYy00MmU4LWI1MWEtMWZhYzk3MWQ3OTdhXzAxMA==    51
2-MjY1MmZmZWMtZWFjYi00MDNiLWFjZTctMTQ4NjdhNjMzYmQwXzAxMg==    46
2-MDBkYjdlOTAtZTVmMS00YmM0LWIzODAtNDhjZDg2ZGQ3NmQ3XzAxMg==    45
2-ZTFhZDUxMzQtZWQ3OC01YmJlLWFhM2QtODZlMjE1YmZlMGJlXzAwMA==    30
2-MGYzODQ3NTgtODQ4ZS00ZDk5LWI2MjQtMTY0MmZhYWQxZGI3XzAxMg==    24
dtype: int64

## Create Conversation Summary

Build a comprehensive summary for each conversation showing:
- The conversation ID
- The sender's name (excluding the user)
- Number of messages from each sender
- Number of messages from the user

This allows us to identify one-sided conversations where the user never replied.


In [6]:
user_name = 'Juan Salas'

# Split into two dataframes
user_messages = messages[messages['FROM'] == user_name]
other_messages = messages[messages['FROM'] != user_name]

# Count messages per conversation for each
user_counts = user_messages.groupby('CONVERSATION ID').size().reset_index(name='user count')
other_counts = other_messages.groupby(['CONVERSATION ID', 'FROM']).size().reset_index(name='from count').rename(columns={'FROM': 'from name'})

# Start with all unique conversation IDs
all_conversations = pd.DataFrame({'CONVERSATION ID': messages['CONVERSATION ID'].unique()})

# Left join user counts
conversation_summary = all_conversations.merge(user_counts, on='CONVERSATION ID', how='left')
conversation_summary['user count'] = conversation_summary['user count'].fillna(0).astype(int)

# Left join other counts (this will create multiple rows for conversations with multiple other parties)
conversation_summary = conversation_summary.merge(other_counts, on='CONVERSATION ID', how='left')
conversation_summary['from count'] = conversation_summary['from count'].fillna(0).astype(int)

# Reorder columns
conversation_summary = conversation_summary[['CONVERSATION ID', 'from name', 'from count', 'user count']].rename(columns={'CONVERSATION ID': 'conversation id'})

conversation_summary.head(20)


,conversation id,from name,from count,user count
0,2-ODEyM2RlMTQtZjFlMC00ODMwLTg1NjItZDkwYTExNTQ3...,LinkedIn Member,1,0
1,2-MzllNzEwNTctNGQ5Yy00NWZkLWI5ZDAtY2ExNjM1NDgx...,Simone Volpicelli,1,0
2,2-ZDBjNTMzNDgtYjJkZC00Yzk4LThmNzktYTNjMjgzN2Q1...,Agus Sánchez - Headhunting and Recruiting Spec...,3,0
3,2-N2YyNzI4MjQtYjFhOC00ODM0LWI0MDAtNTdhODY3NWRm...,Neha Punetha,2,0
4,2-ZGVhNjZkMjYtNWU4ZS00ZjIzLTk5NWUtMTJiOWExMDZm...,Darius Zagrean,6,11
5,2-YmIyNGNlNTEtM2MxOS00YTMxLThkMjItYzAzOWQ2MmM1...,"Alex Harwig, PhD",4,0
6,2-YTIxNDgzM2ItNDczOC00NjliLTg0M2ItZjhlZTZlMWYz...,Rakshit Toke,16,0
7,2-ZWU2NjZlYjItOGJkMS00ODEzLWE3MzctZTFmN2E4ODQ3...,LinkedIn Member,1,0
8,2-NzIyZDI0NTctMGQwZi00NzBlLTlmM2EtZTlkMDlmYzIz...,Maria Taussig,1,0
9,2-NjkyMzRkYzQtNjk5Ni00YjFhLWE0MmEtOWJjYmJmYWUx...,Hanna Nguyen,2,0


## Filter Unreplied Conversations

Filter the conversation summary to only include conversations where:
- The user never replied (user count = 0)

Sort by the number of messages sent to identify the most persistent unreplied senders. These are candidates for connection removal.


In [7]:
# Filter out conversation where user replied
filtered_summary = conversation_summary[conversation_summary['user count'] == 0]

# Sort by conversation id and from count
filtered_summary = filtered_summary.sort_values(['from count'], ascending=[False])

filtered_summary.info()
filtered_summary.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 3369 entries, 33 to 3738
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation id  3369 non-null   object
 1   from name        3369 non-null   object
 2   from count       3369 non-null   int64 
 3   user count       3369 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 131.6+ KB


,conversation id,from name,from count,user count
33,2-MDBkYjdlOTAtZTVmMS00YmM0LWIzODAtNDhjZDg2ZGQ3...,Andrés Alberto Buitrago Alzate,35,0
26,2-MGYzODQ3NTgtODQ4ZS00ZDk5LWI2MjQtMTY0MmZhYWQx...,Mic Mell,24,0
1834,2-ZWQ5MjI4YjctYzg2Zi01MTc0LWI1YTEtYTRmMDE5OGUy...,Andrey Sanenko,21,0
830,2-YjFlZGI4MGQtZDQ4Yy00MjZjLWI3ZjItOWY2MzdlMmU4...,Abhilash Reddy,16,0
6,2-YTIxNDgzM2ItNDczOC00NjliLTg0M2ItZjhlZTZlMWYz...,Rakshit Toke,16,0
1817,2-MTg0MTU2NjgtZjBiZi00OWFiLWI2ZmQtZDE0NjY4MDAy...,Kateryna Slepnova,16,0
85,2-Y2U1YmM0MGMtNmNjMy00MzU3LThmODktN2JiNjE3ZDMw...,Anh Nguyet Ngo,15,0
755,2-ODdjOWMwZGUtMzNkMy00OTI0LWExODktOTk3ZGIwNzZj...,Cora Villa,15,0
688,2-ZDRmMThjZjUtYTUxMS00Yjk0LTliMTctYTM2ZDQwMzI3...,Sunil Chavan,15,0
11,2-MTViNGY0YzYtZjcxMS00MTRiLWFiYWMtZjc4Njk2MDY1...,Aashish Chauhan,14,0


## Join with Connections and Apply Final Filters

Match the filtered senders with their LinkedIn profile URLs from the connections data:
1. Create full names by combining first and last names in the connections dataset
2. Join the unreplied messages data with connections data using names
3. Filter out entries without a matching URL (not in connections list)
4. Apply minimum threshold: only include senders with 3+ unreplied messages

This produces the final list of connections to consider removing.


In [8]:
# Join first and last name in the Connections dataaset
connections['full name'] = connections['First Name'] + ' ' + connections['Last Name']

# Join messages from name with connection url
with_url = filtered_summary[['from name', 'from count']].merge(connections[['full name', 'URL']], left_on='from name', right_on='full name', how='left')
with_url = with_url.drop(columns=['full name'])

# Filter out rows where url is null
with_url = with_url[with_url['URL'].notna()]

# Filter out rows where from count is less than 3
with_url = with_url[with_url['from count'] >= 3]

with_url.info()
with_url.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 589 entries, 0 to 700
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   from name   589 non-null    object
 1   from count  589 non-null    int64 
 2   URL         589 non-null    object
dtypes: int64(1), object(2)
memory usage: 18.4+ KB


,from name,from count,URL
0,Andrés Alberto Buitrago Alzate,35,https://www.linkedin.com/in/andres-buitrago
1,Mic Mell,24,https://www.linkedin.com/in/micmell
3,Abhilash Reddy,16,https://www.linkedin.com/in/mandhadaabhilash
4,Rakshit Toke,16,https://www.linkedin.com/in/toke-tatvasoft
5,Kateryna Slepnova,16,https://www.linkedin.com/in/kateryna-slepnova-...
6,Anh Nguyet Ngo,15,https://www.linkedin.com/in/anhnguyettt
7,Cora Villa,15,https://www.linkedin.com/in/cora-villa-413430168
8,Sunil Chavan,15,https://www.linkedin.com/in/sunil-chavan-1bb65...
9,Aashish Chauhan,14,https://www.linkedin.com/in/aashish-chauhan-17...
10,Ramsinh Manek,13,https://www.linkedin.com/in/ramsinh-manek


## Save Results

Export the final list to a CSV file for further review or automated processing. The output contains:
- Sender name
- Number of unreplied messages
- LinkedIn profile URL


In [9]:
# Save dataframe to csv
with_url.to_csv('data/output.csv', index=False)
